In [28]:
import numpy as np
import cv2
import copy
import math
from scipy.interpolate import interp1d

In [2]:
fname = 'image_mask0.npy'

In [3]:
x = np.load(fname)

In [17]:
y = copy.deepcopy(x[0])

In [5]:
max([max([x for x in y[i] if x < 4]) for i in range(256)])
# [0, 1, 4, 5, 7, 8, 9, 10, 12, 14]

1.0

In [41]:
np.savetxt('corrupted.csv', y, delimiter=',')

In [13]:
masks = []
for val in [0, 1, 4, 5, 7, 8, 9, 10, 12, 14]:
    masks.append(np.array(y == val).astype(np.uint8))

In [30]:
# from https://github.com/neheller/labels18/blob/master/preprocessing/tools.py

class Vector2():
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __add__(self, other):
        return Vector2(self.x + other.x, self.y + other.y)
    def __sub__(self, other):
        return Vector2(self.x - other.x, self.y - other.y)
    def __mul__(self, other):
        return Vector2(self.x * other, self.y * other)

    def magnitude(self):
        return (self.x**2 + self.y**2) ** 0.5

    def unitV(self):
        if self.magnitude() == 0:
            return Vector2(0,0)
        return Vector2(self.x / self.magnitude(), self.y / self.magnitude())

    def dotP(self, other):
        return (self.x * other.x) + (self.y * other.y)

    @staticmethod
    def randomVector():
        return Vector2(random.uniform(-1, 1), random.uniform(-1, 1))

    def __str__(self):
        return "x = " + str(self.x) + " y = " + str(self.y)

def smooth_function(center, variance):
    x = np.linspace(0, 2*math.pi, num = 6, endpoint = True)
    y = np.random.normal(center, variance**0.5, 5)
    y = np.append(y, y[0])
    return interp1d(x, y, kind='cubic')

def perturb_nat(image, variance):
    center = 0
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #these are only needed for RBG images
    # ret, thresh = cv2.threshold(gray, 20, 255, 0)
    contours, hierarchy = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    output = np.zeros(np.shape(image))
    if len(contours) == 0: #blank image
        return output

    for contour in contours:
        f = smooth_function(center, variance)
        size = len(contour)

        br = cv2.boundingRect(contour)
        cx = br[0] + br[2] / 2
        cy = br[1] + br[3] / 2
        center_point = Vector2(cx, cy)

        for count, point in enumerate(contour):
            x = (count / size) * 2 * math.pi
            distance_to_move = f(x)
            current_point = Vector2(point[0][0], point[0][1])
            direction = current_point - center_point
            direction = direction.unitV()
            offset = direction * distance_to_move
            contour[count][0] = [current_point.x - offset.x, current_point.y - offset.y]

        cv2.fillPoly(output, pts=[contour], color=1)
    return output

In [35]:
variance = 10
for val in [0, 1, 4, 5, 7, 8, 9, 10, 12, 14]:
    mask = np.array(y == val).astype(np.uint8)
    output = perturb_nat(mask, variance).astype(np.uint8)
    y[output] = val

0.0078125